In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error

class AdagradRegression:
    def __init__(self, lr=0.01, eps=1e-8):
        self.lr = lr
        self.eps = eps
        self.W = None
        self.G = None

    def fit(self, X, y):
        # Standardize the data
        scaler = StandardScaler()
        X_std = scaler.fit_transform(X)
        y_std = scaler.fit_transform(y.values.reshape(-1,1))

        # Initialize the weights
        n_features = X_std.shape[1]
        self.W = np.zeros((n_features, 1))

        # Initialize the gradient
        self.G = np.zeros((n_features, 1))

        # Train the model
        for i in range(X_std.shape[0]):
            x = X_std[i, :].reshape(n_features, 1)
            y_true = y_std[i, :]

            # Compute the gradient
            grad = np.dot(x, (np.dot(x.T, self.W) - y_true))
            self.G += grad**2

            # Update the weights
            self.W -= self.lr * grad / (np.sqrt(self.G) + self.eps)

    def predict(self, X):
        # Standardize the data
        scaler = StandardScaler()
        X_std = scaler.fit_transform(X)

        # Make predictions
        y_pred = np.dot(X_std, self.W)

        # Inverse transform the predictions
        y_pred = scaler.inverse_transform(y_pred)

        return y_pred.flatten()

# Load data
train_df = pd.read_csv('/kaggle/input/ml-datasets/train.csv')
test_df = pd.read_csv('/kaggle/input/ml-datasets/test.csv')

# Select features
X_train = train_df[['PRODUCT_TYPE_ID']]
X_test = test_df[['PRODUCT_TYPE_ID']]
y_train = train_df['PRODUCT_LENGTH']
y_test = test_df[['PRODUCT_TYPE_ID']]

# Train Adagrad Regression model
ada_reg = AdagradRegression(lr=0.01, eps=1e-8)
ada_reg.fit(X_train, y_train)

# Make predictions on test data
y_pred = ada_reg.predict(X_test)
score = max(0, 100*(1-mean_absolute_percentage_error(y_test, y_pred)))
print(score)

# Create submission file
sub_df = pd.DataFrame({'PRODUCT_ID': test_df['PRODUCT_ID'], 'PRODUCT_LENGTH': y_pred})
sub_df.set_index('PRODUCT_ID', inplace=True)
sub_df.to_csv('Adagrad Regression Result Submission.csv')


0
